In [1]:
! pip install langchain==0.2.2
! pip install langchain-community==0.2.3
! pip install langchain-openai==0.1.8

In [2]:
deployment = "gpt-4o"
model = "gpt-4o"

# 说明

使用 langchain 之后，这两个环境变量要重命名一下，否则会报错。

```bash
OPENAI_API_BASE -> AZURE_OPENAI_ENDPOINT
OPENAI_API_KEY -> AZURE_OPENAI_API_KEY
```

注意，是重命名，不能并存，否则会报错。

重命名后，不影响 OpenAI API 的使用，即重命名前后，都可以正常使用。

# 提示词模板

In [3]:
import os
from langchain import PromptTemplate, LLMChain

# from langchain_openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI

# from langchain.chat_models import ChatOpenAI # 直接访问 OpenAI 的 API 

prompt_template = """Make a good name for a product, the product is described as follow: {product}. Please only return the best one. """

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务


# llm = AzureChatOpenAI(azure_deployment=deployment) # OK
llm = AzureChatOpenAI(model=deployment,
                      temperature=0, max_tokens=200)

llm_chain = LLMChain(prompt=PromptTemplate.from_template(prompt_template), llm=llm)
llm_chain.invoke("colorful socks")
# llm_chain.run("colorful socks")

/Users/gevin/projects/pycharm/ai/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'product': 'colorful socks', 'text': 'VividSoles'}

In [4]:
products = [{"product": "'cloudnative devops platform'"},
            {"product": "'Noise cancellation headphone'"},
            {"product": "colorful socks"}]
llm_chain.apply(products)

[{'text': 'CloudCraft DevOps Platform'},
 {'text': 'SilentWave Pro'},
 {'text': 'VividSoles'}]

# API调用链
## HTTP request chain

In [7]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.llm_requests import LLMRequestsChain
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=deployment, temperature=0, max_tokens=200)


def query_baidu(question: str):
    # template = """
    # You are a search engine.
    # You are given a question, please search the internet and return the answer.
    # Question: {question}
    # Answer:
    # """
    template = """
      Between >>> and <<< are the raw search result text from web.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:
    """
    prompt = PromptTemplate(
        input_variables=["query", "requests_result"],
        template=template,
    )
    
    inputs = {
          "query": question,
          "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      }
    
    chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt),
                              output_key="query_info", verbose=True)
    res = chain.invoke(inputs)
    return res
    
    

In [8]:
result = query_baidu("今天杭州天气")
print("result: ", result)



> Entering new LLMRequestsChain chain...

> Finished chain.
result:  {'query': '今天杭州天气', 'url': 'http://www.baidu.com/s?wd=今天杭州天气', 'query_info': 'Extracted: 20°阴，东北风1级，17~23°C'}
